In [ ]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv").dropna()
df = df[(df.league=='Barclays Premier League') | (df.league=='English League Championship') | (df.league=='Major League Soccer')]
df['spi_margin'] = np.abs(df['spi1'] - df['spi2'])
df['margin'] = np.abs(df['score1'] - df['score2'])
df['totgoals'] = df['score1'] + df['score2']

res = []

for x in df['totgoals']:
  if x > 0:
    res.append("GOALS")
  else:
    res.append("SCORELESS")

df['goals'] = res

In [ ]:
def load_model():
  model = Sequential()
  model.add(Dense(16, activation='relu'))
  model.add(Dense(2, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

def to_cat(array):
  y = to_categorical(np.asarray(pd.factorize(array)[0]))
  return y

def to_df(array):
  df = pd.DataFrame(array).reset_index(drop=True)
  return df

In [ ]:
model = load_model()

In [ ]:
X = df[['score1','score2']].reset_index(drop=True)
y = df[['goals']].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=6)

In [ ]:
y_train = to_cat(y_train['goals'])
y_train = to_df(y_train)
y_test = to_cat(y_test['goals'])
y_test = to_df(y_test)

In [ ]:
history = model.fit(X_train, y_train, epochs=40, batch_size=128, verbose=1, validation_split=0.1)

Epoch 1/40
29/29 [==============================] - 1s 23ms/step - loss: 0.7953 - accuracy: 0.4563 - val_loss: 0.6067 - val_accuracy: 0.6788
Epoch 2/40
29/29 [==============================] - 0s 3ms/step - loss: 0.5550 - accuracy: 0.7294 - val_loss: 0.4314 - val_accuracy: 0.9489
Epoch 3/40
29/29 [==============================] - 0s 4ms/step - loss: 0.4087 - accuracy: 0.9264 - val_loss: 0.3286 - val_accuracy: 0.9489
Epoch 4/40
29/29 [==============================] - 0s 4ms/step - loss: 0.3167 - accuracy: 0.9340 - val_loss: 0.2625 - val_accuracy: 0.9489
Epoch 5/40
29/29 [==============================] - 0s 4ms/step - loss: 0.2646 - accuracy: 0.9299 - val_loss: 0.2189 - val_accuracy: 0.9489
Epoch 6/40
29/29 [==============================] - 0s 4ms/step - loss: 0.2303 - accuracy: 0.9220 - val_loss: 0.1893 - val_accuracy: 0.9489
Epoch 7/40
29/29 [==============================] - 0s 3ms/step - loss: 0.1984 - accuracy: 0.9306 - val_loss: 0.1675 - val_accuracy: 0.9489
Epoch 8/40
29/29 [=

In [ ]:
predictions = model.predict(X_test).round(0)
pred2 = model.predict(X_train)

In [ ]:
loss2, acc2 = model.evaluate(y_train, pred2)
loss, acc = model.evaluate(y_test, predictions)

33/33 [==============================] - 0s 1ms/step - loss: 0.0649 - accuracy: 1.0000


In [ ]:
print("Train Accuracy: %.2f" % (acc2 * 100) + "%")
print("Test Accuracy: %.2f" % (acc * 100) + "%")

Train Accuracy: 100.00%
Test Accuracy: 100.00%
